# Data Visualization Redesign Project - Derek Thomas - 03Nov2017

## Define the environment for the Dataframe

In [43]:
import pandas as pd
import numpy as np
import urllib3
http = urllib3.PoolManager()
import string
import re
import json
from collections import OrderedDict
import warnings
warnings.filterwarnings('ignore')

## Import the College Rankings .csv file and create initial Dataframe

In [44]:
df = pd.read_csv('Data-Visualization-Redesign.csv', header=0,)
df.head(4)

,Order,College Name,City and State,Money Ranking,Median SAT/ACT score,Est. price 2017-18 without aid,Est. price 2017-18 with avg. grant,% who get need-based grants,% who get merit grants,Business major early career earnings
0,1.0,University of California-Berkeley,"Berkeley, CA",4.0,1370/32,"$37,220","$17,929",48%,1%,"$72,100"
1,2.0,Washington and Lee University,"Lexington, VA",16.0,1390/32,"$65,737","$23,089",41%,10%,"$60,000"
2,3.0,University of Pennsylvania,"Philadelphia, PA",27.0,1460/33,"$71,074","$25,838",47%,NaN,"$69,200"
3,4.0,Brown University,"Providence, RI",34.0,1440/32,"$69,809","$24,794",44%,NaN,"$64,100"


## Import the referenced raw Dept of Education source data

### Identify the Degree Concentration areas and save to the variable "degreeColumns"

In [45]:
degreeColumns = ['PCIP01', # Agriculture, Agriculture Operations, and Related Sciences
                 'PCIP03', # Natural Resources and Conservation
                 'PCIP04', # Architecture and Related Services
                 'PCIP05', # Area, Ethnic, Cultural, Gender, and Group Studies
                 'PCIP09', # Communication, Journalism, and Related Programs
                 'PCIP10', # Communications Technologies/Technicians and Support Services
                 'PCIP11', # Computer and Information Sciences and Support Services
                 'PCIP12', # Personal and Culinary Services
                 'PCIP13', # Education
                 'PCIP14', # Engineering
                 'PCIP15', # Engineering Technologies and Engineering-Related Fields
                 'PCIP16', # Foreign Languages, Literatures, and Linguistics
                 'PCIP19', # Family and Consumer Sciences/Human Sciences
                 'PCIP22', # Legal Professions and Studies
                 'PCIP23', # English Language and Literature/Letters
                 'PCIP24', # Liberal Arts and Sciences, General Studies and Humanities
                 'PCIP25', # Library Science
                 'PCIP26', # Biological and Biomedical Sciences
                 'PCIP27', # Mathematics and Statistics
                 'PCIP29', # Military Technologies and Applied Sciences
                 'PCIP30', # Multi/Interdisciplinary Studies
                 'PCIP31', # Parks, Recreation, Leisure, and Fitness Studies
                 'PCIP38', # Philosophy and Religious Studies
                 'PCIP39', # Theology and Religious Vocations
                 'PCIP40', # Physical Sciences
                 'PCIP41', # Science Technologies/Technicians
                 'PCIP42', # Psychology
                 'PCIP43', # Homeland Security, Law Enforcement, Firefighting and Related Protective Services
                 'PCIP44', # Public Administration and Social Service Professions
                 'PCIP45', # Social Sciences
                 'PCIP46', # Construction Trades
                 'PCIP47', # Mechanic and Repair Technologies/Technicians
                 'PCIP48', # Precision Production
                 'PCIP49', # Transportation and Materials Moving
                 'PCIP50', # Visual and Performing Arts
                 'PCIP51', # Health Professions and Related Programs
                 'PCIP52', # Business, Management, Marketing, and Related Support Services
                 'PCIP54'] # History
          

### Read in source csv value, only retrieving applicable columns from data set

In [46]:
dfSchools = pd.read_csv('MERGED2015_16_PP.csv',
                header=0,
                usecols=(['UNITID',    # Unit ID for institution
                        'INSTNM',      # Institution name
                        'CITY',        # City
                        'STABBR',      # State postcode
                        'ZIP',         # ZIP code
                        'HIGHDEG',     # Highest degree awarded: 0-Non Degree;1-Certificate;2-Associate;3-Bachelor's;4-Graduate
                        'CONTROL',     # Control of institution: 1-Public;2-Private nonprofit;3-Private for-profit
                        'REGION',      # Region: 1-New England;2-Mid Ease;3-Great Lakes;4-Plains;5-Southease;6-Southwest;7-Rocky Mountains;8-Far West;9-Outlying Areas
                        'LATITUDE',    # Latitude (Decimal)
                        'LONGITUDE',   # Longitude (Decimal)
                        'CCUGPROF',    # Carnegie Classification - undergraduate profile: >= 5 for Four-year programs
                        'ADM_RATE',    # Admission Rate
                        'ACTCMMID',    # Midpoint of the ACT cumulative score
                        'SAT_AVG'] +   # Average SAT equivalent score of students admitted
                        degreeColumns +# List of Percentage of degrees awarded for all majors
                        ['CIP52BACHL',  # Bachelor's degree in Business, Management, Marketing, And Related Support Services
                        'COSTT4_A',    # Average cost of attendance (academic year institutions)
                        'COSTT4_P',    # Average cost of attendance (program-year institutions)
                        'PCTPELL']))   # Percentage of undergraduates who receive a Pell Grant
dfSchools.head(2)

,UNITID,INSTNM,CITY,STABBR,ZIP,HIGHDEG,CONTROL,REGION,LATITUDE,LONGITUDE,...,PCIP48,PCIP49,PCIP50,PCIP51,PCIP52,PCIP54,CIP52BACHL,COSTT4_A,COSTT4_P,PCTPELL
0,100654,Alabama A & M University,Normal,AL,35762,4,1,5,34.783368,-86.568502,...,0.0,0.0,0.0258,0.0000,0.1479,0.0000,2.0,20809.0,NaN,0.7249
1,100663,University of Alabama at Birmingham,Birmingham,AL,35294-0110,4,1,5,33.502230,-86.809170,...,0.0,0.0,0.0376,0.2231,0.1837,0.0188,1.0,22232.0,NaN,0.3505


In [47]:
len(dfSchools) # Total number of records prior to filtering

7593

### Calculate percentage of Business Degrees awarded compared to all others for 4-year Undergraduate programs

### Rename data set columns identifying the "Percentage of Degrees" awarded for each major concentration area

In [48]:
dfDegreeData = pd.DataFrame([{'DegID' : 'PCIP01', 'Deg Name' : 'Agriculture, Agriculture Operations, and Related Sciences'},
                {'DegID' : 'PCIP03', 'Deg Name' : 'Natural Resources and Conservation'},
                {'DegID' : 'PCIP04', 'Deg Name' : 'Architecture and Related Services'},
                {'DegID' : 'PCIP05', 'Deg Name' : 'Area, Ethnic, Cultural, Gender, and Group Studies'},
                {'DegID' : 'PCIP09', 'Deg Name' : 'Communication, Journalism, and Related Programs'},
                {'DegID' : 'PCIP10', 'Deg Name' : 'Communications Technologies/Technicians and Support Services'},
                {'DegID' : 'PCIP11', 'Deg Name' : 'Computer and Information Sciences and Support Services'},
                {'DegID' : 'PCIP12', 'Deg Name' : 'Personal and Culinary Services'},
                {'DegID' : 'PCIP13', 'Deg Name' : 'Education'},
                {'DegID' : 'PCIP14', 'Deg Name' : 'Engineering'},
                {'DegID' : 'PCIP15', 'Deg Name' : 'Engineering Technologies and Engineering-Related Fields'},
                {'DegID' : 'PCIP16', 'Deg Name' : 'Foreign Languages, Literatures, and Linguistics'},
                {'DegID' : 'PCIP19', 'Deg Name' : 'Family and Consumer Sciences/Human Sciences'},
                {'DegID' : 'PCIP22', 'Deg Name' : 'Legal Professions and Studies'},
                {'DegID' : 'PCIP23', 'Deg Name' : 'English Language and Literature/Letters'},
                {'DegID' : 'PCIP24', 'Deg Name' : 'Liberal Arts and Sciences, General Studies and Humanities'},
                {'DegID' : 'PCIP25', 'Deg Name' : 'Library Science'},
                {'DegID' : 'PCIP26', 'Deg Name' : 'Biological and Biomedical Sciences'},
                {'DegID' : 'PCIP27', 'Deg Name' : 'Mathematics and Statistics'},
                {'DegID' : 'PCIP29', 'Deg Name' : 'Military Technologies and Applied Sciences'},
                {'DegID' : 'PCIP30', 'Deg Name' : 'Multi/Interdisciplinary Studies'},
                {'DegID' : 'PCIP31', 'Deg Name' : 'Parks, Recreation, Leisure, and Fitness Studies'},
                {'DegID' : 'PCIP38', 'Deg Name' : 'Philosophy and Religious Studies'},
                {'DegID' : 'PCIP39', 'Deg Name' : 'Theology and Religious Vocations'},
                {'DegID' : 'PCIP40', 'Deg Name' : 'Physical Sciences'},
                {'DegID' : 'PCIP41', 'Deg Name' : 'Science Technologies/Technicians'},
                {'DegID' : 'PCIP42', 'Deg Name' : 'Psychology'},
                {'DegID' : 'PCIP43', 'Deg Name' : 'Homeland Security, Law Enforcement, Firefighting and Related Protective Services'},
                {'DegID' : 'PCIP44', 'Deg Name' : 'Public Administration and Social Service Professions'},
                {'DegID' : 'PCIP45', 'Deg Name' : 'Social Sciences'},
                {'DegID' : 'PCIP46', 'Deg Name' : 'Construction Trades'},
                {'DegID' : 'PCIP47', 'Deg Name' : 'Mechanic and Repair Technologies/Technicians'},
                {'DegID' : 'PCIP48', 'Deg Name' : 'Precision Production'},
                {'DegID' : 'PCIP49', 'Deg Name' : 'Transportation and Materials Moving'},
                {'DegID' : 'PCIP50', 'Deg Name' : 'Visual and Performing Arts'},
                {'DegID' : 'PCIP51', 'Deg Name' : 'Health Professions and Related Programs'},
                {'DegID' : 'PCIP52', 'Deg Name' : 'Business, Management, Marketing, and Related Support Services'},
                {'DegID' : 'PCIP54', 'Deg Name' : 'History'}])
dfDegreeData = dfDegreeData.set_index('DegID')
dfDegreeData.head(2)

,Deg Name
DegID,
PCIP01,"Agriculture, Agriculture Operations, and Relat..."
PCIP03,Natural Resources and Conservation


In [49]:
MajorTotals = dfSchools[((dfSchools['HIGHDEG'] >= 3) &
                           (dfSchools['CCUGPROF'] >= 6) &
                           (dfSchools['PCIP52'] > 0))]
MajorTotals.head(2)

,UNITID,INSTNM,CITY,STABBR,ZIP,HIGHDEG,CONTROL,REGION,LATITUDE,LONGITUDE,...,PCIP48,PCIP49,PCIP50,PCIP51,PCIP52,PCIP54,CIP52BACHL,COSTT4_A,COSTT4_P,PCTPELL
0,100654,Alabama A & M University,Normal,AL,35762,4,1,5,34.783368,-86.568502,...,0.0,0.0,0.0258,0.0000,0.1479,0.0000,2.0,20809.0,NaN,0.7249
1,100663,University of Alabama at Birmingham,Birmingham,AL,35294-0110,4,1,5,33.502230,-86.809170,...,0.0,0.0,0.0376,0.2231,0.1837,0.0188,1.0,22232.0,NaN,0.3505


In [50]:
len(MajorTotals)

1809

In [51]:
cumMajTotals = MajorTotals[degreeColumns].sum().sort_values(ascending=False)
cumMajTotals

PCIP52    359.6532
PCIP51    282.8241
PCIP13     95.6981
PCIP50     94.7589
PCIP42     94.1417
PCIP26     83.7716
PCIP11     82.6194
PCIP45     77.4936
PCIP43     72.0524
PCIP24     65.4174
PCIP09     58.4406
PCIP15     48.5480
PCIP31     44.2491
PCIP14     43.3545
PCIP23     34.3616
PCIP44     33.5107
PCIP30     29.9669
PCIP39     29.7830
PCIP12     22.2473
PCIP54     21.2933
PCIP40     20.1386
PCIP27     14.6695
PCIP19     13.4795
PCIP03     12.6760
PCIP38     11.5718
PCIP16     10.9742
PCIP01     10.2394
PCIP22      9.1615
PCIP10      6.6495
PCIP47      5.6515
PCIP49      4.8796
PCIP04      4.8419
PCIP05      4.4965
PCIP46      3.0009
PCIP48      1.6792
PCIP29      0.3619
PCIP41      0.3193
PCIP25      0.0250
dtype: float64

In [52]:
dfDegreeData['cumPercentages'] = cumMajTotals
dfDegreeData.head(4)

,Deg Name,cumPercentages
DegID,,
PCIP01,"Agriculture, Agriculture Operations, and Relat...",10.2394
PCIP03,Natural Resources and Conservation,12.6760
PCIP04,Architecture and Related Services,4.8419
PCIP05,"Area, Ethnic, Cultural, Gender, and Group Studies",4.4965


In [53]:
dfDegreeData['ActualPercent'] = (dfDegreeData['cumPercentages'] / dfDegreeData['cumPercentages'].sum())
dfDegreeData.head(4)

,Deg Name,cumPercentages,ActualPercent
DegID,,,
PCIP01,"Agriculture, Agriculture Operations, and Relat...",10.2394,0.005660
PCIP03,Natural Resources and Conservation,12.6760,0.007007
PCIP04,Architecture and Related Services,4.8419,0.002677
PCIP05,"Area, Ethnic, Cultural, Gender, and Group Studies",4.4965,0.002486


In [54]:
MajorTotals['PCIP52'].sum() / (MajorTotals[degreeColumns].sum()).sum()

0.19881313511566498

In [55]:
dfDegreeData.loc['PCIP52','Deg Name']

'Business, Management, Marketing, and Related Support Services'

In [56]:
AllMajors = len(dfSchools[((dfSchools['HIGHDEG'] >= 3) &
                           (dfSchools['CCUGPROF'] >= 6))])
AllMajors

2226

### Filter Schoold data for colleges offering 4-yr Undegraduate Business degrees

In [57]:
dfSchools = dfSchools[((dfSchools['HIGHDEG'] >= 3) &     # Colleges offering Bachelors degrees and higher
 (dfSchools['CCUGPROF'] >= 6) &    # 4 Year Undegraduate programs
(dfSchools['PCIP52'] > 0) &        # % of Business Degrees awarded greater than 0
 (dfSchools['CIP52BACHL'] > 0))]    # Number of Business Bachelors Degrees awarded greater than 0
dfSchools.head(4)

,UNITID,INSTNM,CITY,STABBR,ZIP,HIGHDEG,CONTROL,REGION,LATITUDE,LONGITUDE,...,PCIP48,PCIP49,PCIP50,PCIP51,PCIP52,PCIP54,CIP52BACHL,COSTT4_A,COSTT4_P,PCTPELL
0,100654,Alabama A & M University,Normal,AL,35762,4,1,5,34.783368,-86.568502,...,0.0,0.0,0.0258,0.0000,0.1479,0.0000,2.0,20809.0,NaN,0.7249
1,100663,University of Alabama at Birmingham,Birmingham,AL,35294-0110,4,1,5,33.502230,-86.809170,...,0.0,0.0,0.0376,0.2231,0.1837,0.0188,1.0,22232.0,NaN,0.3505
3,100706,University of Alabama in Huntsville,Huntsville,AL,35899,4,1,5,34.722818,-86.638420,...,0.0,0.0,0.0288,0.1892,0.2072,0.0117,1.0,20999.0,NaN,0.3179
4,100724,Alabama State University,Montgomery,AL,36104-0271,4,1,5,32.364317,-86.295677,...,0.0,0.0,0.0473,0.0926,0.0983,0.0113,1.0,18100.0,NaN,0.7567


In [58]:
len(dfSchools) # Total number of records after filtering

1775

### Modify column names

In [59]:
dfSchools.columns=(['Unit ID',\
            'Institution Name',\
            'City',\
            'State',\
            'Zip Code',\
            'High Degree',\
            'Institution Type',\
            'Region ID',\
            'Latitude',\
            'Longitude',\
            'Program Type',\
            'Admission Rate',\
            'ACT Score',\
            'SAT Average'] +\
            degreeColumns +\
            ['# of Business Degrees',\
            'Attendance Cost - Academic Yr',\
            'Attendance Cost - Program Yr',\
            '% of Pell Grant Awards'])
dfSchools.head(4)

,Unit ID,Institution Name,City,State,Zip Code,High Degree,Institution Type,Region ID,Latitude,Longitude,...,PCIP48,PCIP49,PCIP50,PCIP51,PCIP52,PCIP54,# of Business Degrees,Attendance Cost - Academic Yr,Attendance Cost - Program Yr,% of Pell Grant Awards
0,100654,Alabama A & M University,Normal,AL,35762,4,1,5,34.783368,-86.568502,...,0.0,0.0,0.0258,0.0000,0.1479,0.0000,2.0,20809.0,NaN,0.7249
1,100663,University of Alabama at Birmingham,Birmingham,AL,35294-0110,4,1,5,33.502230,-86.809170,...,0.0,0.0,0.0376,0.2231,0.1837,0.0188,1.0,22232.0,NaN,0.3505
3,100706,University of Alabama in Huntsville,Huntsville,AL,35899,4,1,5,34.722818,-86.638420,...,0.0,0.0,0.0288,0.1892,0.2072,0.0117,1.0,20999.0,NaN,0.3179
4,100724,Alabama State University,Montgomery,AL,36104-0271,4,1,5,32.364317,-86.295677,...,0.0,0.0,0.0473,0.0926,0.0983,0.0113,1.0,18100.0,NaN,0.7567


### Add new column to identify Schools by region
    -New England (CT, ME, MA, NH, RI, VT)
    -Mid East (DE, DC, MD, NJ, NY, PA)
    -Great Lakes (IL, IN, MI, OH, WI)
    -Plains (IA, KS, MN, MO, NE, ND, SD)
    -Southeast (AL, AR, FL, GA, KY, LA, MS, NC, SC, TN, VA, WV)
    -Southwest (AZ, NM, OK, TX)
    -Rocky Mountains (CO, ID, MT, UT, WY)
    -Far West (AK, CA, HI, NV, OR, WA)
    -Outlying Areas (AS, FM, GU, MH, MP, PR, PW, VI)


In [60]:
def assignRegion(x):
    if (x == 1):
        return 'New England'
    elif (x == 2):
        return 'Mid East'
    elif (x == 3):
        return 'Great Lakes'
    elif (x == 4):
        return 'Plains'
    elif (x == 5):
        return 'Southeast'
    elif (x == 6):
        return 'Southwest'
    elif (x == 7):
        return 'Rocky Mountains'
    elif (x == 8):
        return 'Far West'
    elif (x == 9):
        return 'Outlying Areas'

In [61]:
dfSchools['Region'] = np.nan
dfSchools['Region'] = dfSchools['Region ID'].apply(assignRegion)
dfSchools.head(4)

,Unit ID,Institution Name,City,State,Zip Code,High Degree,Institution Type,Region ID,Latitude,Longitude,...,PCIP49,PCIP50,PCIP51,PCIP52,PCIP54,# of Business Degrees,Attendance Cost - Academic Yr,Attendance Cost - Program Yr,% of Pell Grant Awards,Region
0,100654,Alabama A & M University,Normal,AL,35762,4,1,5,34.783368,-86.568502,...,0.0,0.0258,0.0000,0.1479,0.0000,2.0,20809.0,NaN,0.7249,Southeast
1,100663,University of Alabama at Birmingham,Birmingham,AL,35294-0110,4,1,5,33.502230,-86.809170,...,0.0,0.0376,0.2231,0.1837,0.0188,1.0,22232.0,NaN,0.3505,Southeast
3,100706,University of Alabama in Huntsville,Huntsville,AL,35899,4,1,5,34.722818,-86.638420,...,0.0,0.0288,0.1892,0.2072,0.0117,1.0,20999.0,NaN,0.3179,Southeast
4,100724,Alabama State University,Montgomery,AL,36104-0271,4,1,5,32.364317,-86.295677,...,0.0,0.0473,0.0926,0.0983,0.0113,1.0,18100.0,NaN,0.7567,Southeast


## Import the referenced raw PayScale.com source data

In [62]:
dfPay = pd.read_csv('PayScale-Data.csv', header=0)
dfPay.head(4)

,OrigOrder,Rank,School Name,School Type,Early Career Pay,Mid-Career Pay,% High Meaning,% STEM Degrees
0,1,1,Harvey Mudd College,"Engineering, Liberal Arts School, Private School","$81,000","$155,800",58%,85%
1,2,2,Princeton University,"Ivy League, Private School, Research Universit...","$69,800","$147,800",44%,47%
2,3,3,Massachusetts Institute of Technology,"Engineering, Private School, Research University","$81,500","$147,000",53%,69%
3,4,4,SUNY Maritime College,State School,"$71,600","$145,100",52%,35%


## Write Data Frames to ".csv" file for import into Tableau

In [63]:
df.to_csv('CollegeRankings.csv')
dfSchools.to_csv('CollegeSchools.csv')
dfPay.to_csv('CollegeJobPay.csv')
dfDegreeData.to_csv('CollegeDegreeData.csv')
dfPay.to_csv('CollegePayScale.csv')